In [8]:
import py_itpp as itpp
import numpy as np

def generate_frequency_dependent_chan(delay_profile, chan_tdom):
    L=delay_profile.shape[0]
    U = lambda f: ((np.exp(-1.0j*2*np.pi*f*delay_profile))*(chan_tdom)).sum(axis=0)
    return U

class MIMO_TDL_Channel():
    def __init__(self,Nt,Nr,c_spec,sampling_time):
        self.Nt=Nt
        self.Nr=Nr
        self.sampling_time = sampling_time
        self.tdl_channels=[]
        #Declare Nt*Nr channels
        avg_power_db = itpp.vec()
        tap_delay = itpp.ivec()
        channel1 = itpp.TDL_Channel(c_spec, sampling_time)
        channel1.get_channel_profile(avg_power_db, tap_delay)
        self.channel_length=channel1.taps()
        # print 'Channel Profile'
        # print 'Avg power [dB]', '\tDelay [samples]'
        # for tap_index in range(self.channel_length):
        #     print avg_power_db[tap_index], '\t', tap_delay[tap_index]
        for i in range(self.Nt*self.Nr):
            self.tdl_channels.append(itpp.TDL_Channel(c_spec,self.sampling_time))

    def generate(self):
        self.py_channel=[]
        self.channel=itpp.cmat()
        self.channel.set_size(self.Nt*self.Nr,self.channel_length,False)
        for i in range(self.channel_length):
            chan_instance=np.zeros(shape=(self.Nt,self.Nr),dtype=complex)
            self.py_channel.append(chan_instance)

        channel_coef_one=itpp.cmat()

        for i in range(self.Nr):
            for j in range(self.Nt):
                self.tdl_channels[i*self.Nr+j].generate(1,channel_coef_one)
                for l in range(self.channel_length):
                    self.py_channel[l][i][j]=channel_coef_one(0,l)
                    self.channel.set(i*self.Nr+j,l,channel_coef_one(0,l))
        #print(self.py_channel[0])
        return self.py_channel

    def set_norm_doppler(self,norm_doppler):
        for i in range(self.Nt*self.Nr):
            self.tdl_channels[i].set_norm_doppler(norm_doppler)

# Sim Params
macroBSTx=43
femtoBSTx=15
macroUETx=25
femtoUETx=10
D12=1000
R1=200
N0_Hz=-174
macro_loss=lambda R: 128.1+37.6*np.log10(R/1000)
Lv=6.9
femto_loss=lambda R,n: 37 + 32*np.log10(R)+n*Lv
fading_const=8

# Frequency Array
center_frequency=0.0
bandwidth_macro=2e+6
start_frequency_macro=center_frequency-(bandwidth_macro/2)
end_frequency_macro=center_frequency+(bandwidth_macro/2)
bandwidth_femto=1e+6
start_frequency_femto=center_frequency-(bandwidth_femto/2)
end_frequency_femto=center_frequency+(bandwidth_femto/2)
#----------------------------------------------------------------------
# FFT Params
number_bins=64
N_subcarrier_macro=64
bins_left_edges=np.linspace(start_frequency_macro, end_frequency_macro, num=(number_bins+1))[:number_bins]
bin_size=bins_left_edges[1]-bins_left_edges[0]
freq_arr_macro=bins_left_edges+(bin_size/2)
number_bins=32
N_subcarrier_femto=32
bins_left_edges=np.linspace(start_frequency_femto, end_frequency_femto, num=(number_bins+1))[:number_bins]
bin_size=bins_left_edges[1]-bins_left_edges[0]
freq_arr_femto=bins_left_edges+(bin_size/2)
#----------------------------------------------------------------------

chan_power_V=np.array( [0.0e-9,310.0e-9,710.0e-9,1090.0e-9,1730.00e-9,2510.00e-9])
chan_delay_V=np.array([0.0, -1.0, -9.0, -10.0, -15.0, -20.0])
chan_power_P=np.array([0.0e-9,110.0e-9,190.0e-9,410.0e-9])
chan_delay_P=np.array([0,-9.7,-19.2,-22.8])

#=np.array([0,-1.5,-1.4,-3.6,-0.6,-9.1,-7.0,-12.0,-16.9]) #in dB
#=np.array([0,30,150,310,370,710,1090,1730,2510])# in ns
#=np.array([0,-1.0,-2.0,-3.0,-8.0, -17.2,-20.8]) # in dB
#=np.array([0,30,70,90,110,190,410]) #in ns
itpp.RNG_randomize()
c_spec_V=itpp.Channel_Specification(itpp.CHANNEL_PROFILE.ITU_Vehicular_A)
c_spec_P=itpp.Channel_Specification(itpp.CHANNEL_PROFILE.ITU_Pedestrian_A)

channel_length_P=c_spec_P.taps()
chan_P=MIMO_TDL_Channel(1,1,c_spec_P,1e-9)
chan_P.set_norm_doppler(1e-4)
chan_tdom_P=chan_P.generate()

channel_length_V=c_spec_V.taps()
chan_V=MIMO_TDL_Channel(1,1,c_spec_V,1e-9)
chan_V.set_norm_doppler(1e-4)
chan_tdom_V=chan_V.generate()

freq_chan_P=generate_frequency_dependent_chan(chan_delay_P,chan_tdom_P)
freq_chan_V=generate_frequency_dependent_chan(chan_delay_V,chan_tdom_V)

femto_list_P=list(map(freq_chan_P, freq_arr_femto))
femto_list_V=list(map(freq_chan_V, freq_arr_femto))
macro_list_P=list(map(freq_chan_P, freq_arr_macro))
macro_list_V=list(map(freq_chan_V, freq_arr_macro))



[array([[-0.40057177+0.24954197j, -0.40057177+0.24954197j,
        -0.40057177+0.24954197j, -0.40057177+0.24954197j,
        -0.40057177+0.24954197j, -0.40057177+0.24954197j]]), array([[-0.40057177+0.24954197j, -0.40057177+0.24954197j,
        -0.40057177+0.24954197j, -0.40057177+0.24954197j,
        -0.40057177+0.24954197j, -0.40057177+0.24954197j]]), array([[-0.40057177+0.24954197j, -0.40057177+0.24954197j,
        -0.40057177+0.24954197j, -0.40057177+0.24954197j,
        -0.40057177+0.24954197j, -0.40057177+0.24954197j]]), array([[-0.40057177+0.24954197j, -0.40057177+0.24954197j,
        -0.40057177+0.24954196j, -0.40057177+0.24954196j,
        -0.40057177+0.24954196j, -0.40057177+0.24954196j]]), array([[-0.40057177+0.24954197j, -0.40057177+0.24954197j,
        -0.40057177+0.24954196j, -0.40057177+0.24954196j,
        -0.40057177+0.24954197j, -0.40057177+0.24954196j]]), array([[-0.40057177+0.24954197j, -0.40057177+0.24954197j,
        -0.40057177+0.24954196j, -0.40057177+0.24954197j